In [1]:
pip install -U datasets huggingface_hub fsspec

In [2]:
pip install peft

In [3]:
pip install accelerate -U

In [34]:
from transformers import AutoModelForCausalLM, AutoTokenizer , TrainingArguments , Trainer , DataCollatorForLanguageModeling

In [5]:
model_name = "bigscience/bloomz-560m"
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/715 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

In [23]:
def generate_text(model , tokenizer , prompt_text , max_tokens):
    prompt_text = tokenizer(prompt_text, return_tensors= "pt")
    output = model.generate(
        input_ids= prompt_text["input_ids"],
        attention_mask= prompt_text["attention_mask"],
        max_length= max_tokens,repetition_penalty=1.5,
        eos_token_id= tokenizer.eos_token_id
        )
    return tokenizer.batch_decode(output,skip_special_tokens= True)

In [24]:
initial_output = generate_text(model,tokenizer,"I want you to act as a logistician. " , 100)
print("Initial model output: ", initial_output)

Initial model output:  ['I want you to act as a logistician.  You will be able to:  Analyze the data']


In [25]:
from datasets import load_dataset

dataset_prompt = "fka/awesome-chatgpt-prompts"

data_prompt = load_dataset(dataset_prompt)
data_prompt = data_prompt.map(lambda x: tokenizer(x["prompt"]) , batched= True)
train_prompts = data_prompt["train"].select(range(30))

README.md:   0%|          | 0.00/339 [00:00<?, ?B/s]

prompts.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/203 [00:00<?, ? examples/s]

Map:   0%|          | 0/203 [00:00<?, ? examples/s]

In [26]:
print(data_prompt['train'][:5])

{'act': ['An Ethereum Developer', 'SEO Prompt', 'Linux Terminal', 'English Translator and Improver', '`position` Interviewer'], 'prompt': ['Imagine you are an experienced Ethereum developer tasked with creating a smart contract for a blockchain messenger. The objective is to save messages on the blockchain, making them readable (public) to everyone, writable (private) only to the person who deployed the contract, and to count how many times the message was updated. Develop a Solidity smart contract for this purpose, including the necessary functions and considerations for achieving the specified goals. Please provide the code and any relevant explanations to ensure a clear understanding of the implementation.', "Using WebPilot, create an outline for an article that will be 2,000 words on the keyword 'Best SEO prompts' based on the top 10 results from Google. Include every relevant heading possible. Keep the keyword density of the headings high. For each section of the outline, include 

In [27]:
import pandas as ps

df = ps.DataFrame(data_prompt['train'][:])
df.head()

,act,prompt,input_ids,attention_mask
0,An Ethereum Developer,Imagine you are an experienced Ethereum develo...,"[186402, 1152, 1306, 660, 72560, 28857, 167625...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,SEO Prompt,"Using WebPilot, create an outline for an artic...","[39312, 15202, 51, 46712, 15, 7932, 660, 67606...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,Linux Terminal,I want you to act as a linux terminal. I will ...,"[44, 4026, 1152, 427, 1769, 661, 267, 104105, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,English Translator and Improver,"I want you to act as an English translator, sp...","[44, 4026, 1152, 427, 1769, 661, 660, 7165, 24...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,`position` Interviewer,I want you to act as an interviewer. I will be...,"[44, 4026, 1152, 427, 1769, 661, 660, 33322, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [29]:
from peft import get_peft_model , PromptTuningConfig , TaskType , PromptTuningInit

tuning_config = PromptTuningConfig(
    task_type = TaskType.CAUSAL_LM,
    prompt_tuning_init = PromptTuningInit.RANDOM,
    num_virtual_tokens=4,
    tokenizer_name_or_path=model_name
)

peft_model = get_peft_model(model , tuning_config)

In [36]:
training_args = TrainingArguments(
    use_cpu=True,
    output_dir="./",
    auto_find_batch_size =True,
    learning_rate=0.005,
    num_train_epochs = 5
)

In [37]:
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=train_prompts,
    data_collator=DataCollatorForLanguageModeling(tokenizer,mlm=False)
)
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mohammedabdulkarimalh (mohammedabdulkarimalh-student) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss


TrainOutput(global_step=20, training_loss=3.6018817901611326, metrics={'train_runtime': 865.7532, 'train_samples_per_second': 0.173, 'train_steps_per_second': 0.023, 'total_flos': 34550802087936.0, 'train_loss': 3.6018817901611326, 'epoch': 5.0})

In [38]:
tuned_output = generate_text(trainer.model,tokenizer,"I want you to act as a logistician. ", 100)
print(f"Tuned model output: { tuned_output}")

Tuned model output: ['I want you to act as a logistician.  You will be responsible for the logistics of your business.']
